In [11]:
import datadotworld as dw
import pandas as pd
import psycopg
import openai

In [12]:
DB_NAME = "ai_product_catalog"
DB_USER = "ai_product_catalog"
DB_PASSWORD = "ai_product_catalog123"
DB_HOST = "127.0.0.1"
DB_PORT = "5432"
DB_CONNECTION_STRING = f"host={DB_HOST} port={DB_PORT} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD}"

CATEGORY_DESC = "Shoes"

PROVIDER_OPENAI = "openai"
PROVIDER_OPENAI_EMBEDDINGS_MODEL = "text-embedding-ada-002"

In [13]:
product_dataset = dw.load_dataset('data-hut/product-data-from-nike')
product_dataset.describe()

{'name': 'data-hut_product-data-from-nike',
 'title': 'Product data from Nike',
 'description': 'Product data from Nike to help companies and researchers conduct analysis on pricing, product, and assortment.\n\n---\n{"editor":"markdown"}\n---\n\n\nProduct data from Nike from helps companies conduct competition analysis , pricing analyisis . Contact Datahut ( [https://datahut.co/](https://datahut.co/) ) for more details.\n\n\u200b\n\nHere is the link to the competitive analysis we conducted using this data set.\n\n\n@(https://blog.datahut.co/competitive-analysis-nike-vs-adidas/)\n\n\n\u200b',
 'homepage': 'https://data.world/data-hut/product-data-from-nike',
 'license': 'Public Domain',
 'resources': [{'name': 'nike_2020_04_13',
   'path': 'data/nike_2020_04_13.csv',
   'format': 'csv',
   'profile': 'data-resource'},
  {'name': 'original/nike_2020_04_13.csv',
   'path': 'original/nike_2020_04_13.csv',
   'format': 'csv',
   'mediatype': 'text/csv',
   'bytes': 791480,
   'profile': 'da

In [14]:
df = product_dataset.dataframes["nike_2020_04_13"]
df.head()

/opt/anaconda3/envs/ai-product-catalog/lib/python3.12/site-packages/datadotworld/models/dataset.py:198: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pandas.read_csv(


,url,product_name,product_id,listing_price,sale_price,discount,brand,description,rating,reviews,images,last_visited
0,https://www.nike.com/in/t/air-force-1-07-essen...,Nike Air Force 1 '07 Essential,CJ1646-600,0,7495,0,Nike,Let your shoe game shimmer in the Nike Air For...,0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:56
1,https://www.nike.com/in/t/air-force-1-07-shoe-...,Nike Air Force 1 '07,CT4328-101,0,7495,0,Nike,The legend lives on in the Nike Air Force 1 '0...,0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:56
2,https://www.nike.com/in/t/air-force-1-sage-low...,Nike Air Force 1 Sage Low LX,CI3482-200,0,9995,0,Nike,"Taking both height and craft to new levels, th...",0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:56
3,https://www.nike.com/in/t/air-max-dia-se-shoe-...,Nike Air Max Dia SE,CD0479-200,0,9995,0,Nike,"Designed for a woman's foot, the Nike Air Max ...",0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:57
4,https://www.nike.com/in/t/air-max-verona-shoe-...,Nike Air Max Verona,CZ6156-101,0,9995,0,Nike,Pass on the good vibes in the Nike Air Max Ver...,0.0,0,"[""https://static.nike.com/a/images/t_PDP_1728_...",2020-04-13 15:27:57


In [15]:
print ("Shape = " + str(df.shape))
print ("Number of unique Product IDs = " + str(len(df['product_id'].drop_duplicates())))
print ("Maximum Length of Product ID Column = " + str(df['product_id'].str.len().max()))
print ("Number of unique Brands = " + str(len(df['brand'].drop_duplicates())))
print ("Maximum Length of Product Name Column = " + str(df['product_name'].str.len().max()))
print ("Maximum Length of Product Description Column = " + str(df['description'].str.len().max()))
print ("Maximum Length of Brand Column = " + str(df['brand'].str.len().max()))


Shape = (643, 12)
Number of unique Product IDs = 554
Maximum Length of Product ID Column = 10
Number of unique Brands = 1
Maximum Length of Product Name Column = 61
Maximum Length of Product Description Column = 532.0
Maximum Length of Brand Column = 4


In [22]:
category_id = None

with psycopg.connect(DB_CONNECTION_STRING) as db_connection:
  with db_connection.cursor() as c:
      c.execute(
          """
              INSERT INTO categories(category_desc) VALUES (%s) RETURNING category_id;
          """, 
          (CATEGORY_DESC,))

      rows = c.fetchone()
      if rows:
          category_id = rows[0]

      db_connection.commit()

if category_id == None:
  print ("Issue with category creation.  Was not created.")
else:
  print ("Category Created.  Category_ID = " + str(category_id) + " for Category_Name = " + CATEGORY_DESC)

Category Created.  Category_ID = 7 for Category_Name = Shoes


In [30]:
brandsDF = pd.DataFrame({ "Brand": df['brand'].drop_duplicates() })
print(brandsDF.shape)
brandsDF.head()

(1, 1)


,Brand
0,Nike


In [38]:
with psycopg.connect(DB_CONNECTION_STRING) as db_connection:
  with db_connection.cursor() as c:
    c.executemany(
          """
              INSERT INTO brands ( brand_desc ) 
              VALUES ( %(Brand)s )
              RETURNING brand_id
          """,
        brandsDF.to_dict(orient="records")
    )

In [39]:
df['msrp'] = df['sale_price'].astype('float') / 100.0
df['msrp']

0       74.95
1       74.95
2       99.95
3       99.95
4       99.95
        ...  
638    127.97
639     34.97
640     59.47
641    169.95
642     62.97
Name: msrp, Length: 643, dtype: float64

In [44]:
with psycopg.connect(DB_CONNECTION_STRING) as db_connection:
  with db_connection.cursor() as c:
    c.executemany(
       f"""
            INSERT INTO products
            (
                sku, 
                brand_id,
                product_name,
                product_desc,
                size,
                msrp,
                category_id
            ) 
            VALUES 
            (
                  %(product_id)s,
                  (select brand_id from brands where brand_desc=%(brand)s fetch first 1 rows only),
                  %(product_name)s,
                  %(description)s,
                  null,
                  cast(%(msrp)s as double precision),
                  {category_id}
            )
            RETURNING product_id
        """,
        df.to_dict(orient="records")
    )
      
    productIds = c.fetchall()
    productIds


ProgrammingError: no result available

In [ ]:
def get_embedding_from_db(productName, sku, price, brand, description, model):
    productName = productName.replace("'", "''")
    
    cleansedDescription = None
    if description != None and type(description) != float:
        cleansedDescription = description.replace("\n", " ").replace("'", "''")

    with psycopg.connect(DB_CONNECTION_STRING) as db_connection:
        with db_connection.cursor() as c:
            sql = f"""
                            select embedding
                            from product_embeddings
                            where model = '{model}'
                              and engine = '{PROVIDER_OPENAI}'
                              and product_id = 
                                (
                                    select product_id 
                                    from products 
                                    where product_name='{productName}'
                                      and sku='{sku}'
                                      and brand_id=(select brand_id from brands where brand_desc='{brand}')
                                      and product_desc"""
            if cleansedDescription != None:
              sql = sql + f"='{cleansedDescription}'"
            else:
              sql = sql + " is null"
            sql = sql + f"""
                                )
                      """
            print(sql)
            
            c.execute(sql)
            record = c.fetchone()

            if record == None:
                  return None
            
            return record[0]

In [ ]:
client = openai.OpenAI()

counter = 0

def create_embedding(productName, sku, price, brand, description, model):
   global counter

   text = """'{productName}', '{sku}', {price}, '{brand}', '{description}'"""
   text = text.replace("\n", " ")

   embedding = get_embedding_from_db(productName, sku, price, brand, description, model)
   
   if embedding != None:
      return None

   counter = counter + 1
   if counter > 1:
      return None

   return client.embeddings.create(input = [text], model=model).data[0].embedding

df['embeddingToStore'] = df.apply(lambda row: create_embedding(row["product_name"], row["product_id"], row["MRPFloat"], row["brand"], row["description"], model=PROVIDER_OPENAI_EMBEDDINGS_MODEL), axis=1)

In [ ]:
embeddingsDF = df[df['embeddingToStore'].notnull()]
print("Shape:", embeddingsDF.shape)
print ("Vector Length:", str(len(embeddingsDF["embeddingToStore"][0])))
embeddingsDF.head()

In [ ]:
with psycopg.connect(DB_CONNECTION_STRING) as db_connection:
  with db_connection.cursor() as c:
      c.executemany(
          f"""
              INSERT INTO product_embeddings
              (
                product_id,
                engine,
                model,
                embedding
              ) 
              VALUES
              (
                  (
                      select product_id 
                      from products 
                      where product_name=%(product_name)s
                        and sku=%(product_id)s
                        and brand_id=(select brand_id from brands where brand_desc=%(brand)s)
                        and product_desc=%(description)s
                  ),
                  '{PROVIDER_OPENAI}',
                  '{PROVIDER_OPENAI_EMBEDDINGS_MODEL}',
                  %(embedding)s
              )
          """,
          embeddingsDF.to_dict(orient="records"),
      )